In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib
#%matplotlib notebook
#%matplotlib qt5
%matplotlib inline
from matplotlib import pyplot as plt
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0) # bigger figures

import seaborn as sns
sns.set() # better looking figs

# hide warnings for clarity
import warnings
warnings.filterwarnings('ignore')

# our own set of small helper functions for plotting, etc
from utils import plot_embedding, plot_compare_embeddings, show_heatmap, plot_confusion_matrix
import factor_analyzer

Load dataset, rename and re-order columns where necessary.

In [ ]:
data = pd.read_csv("../Data/Input/fulldata.csv")
data["clipId"] = data["clipName"].apply(lambda x: x[-8:-6])

# re-order columns + keep only useful ones
data = data[['pptID','fileName', 'condition', 'age', 'gender', 'nationality', 'firstLang', 'trial', 'clipId', 'freetext',
 'q01', 'q02', 'q03', 'q04', 'q05', 'q06', 'q07', 'q08', 'q09', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17',
 'q18', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24', 'q25', 'q26', 'q27', 'q28', 'q29', 'q30']]

Rename `qXX` columns with the names of the actual constructs tested in the questionnaire.

Notes:
- `condition=1` is the 'Movement-only' condition, `condition=2` is the 'Full-scene' condition
- each participant `pptID` watched 4 different clips, hence 4 rows per participant

In [ ]:
constructs=["Sad", "Happy", "Angry", "Excited", "Calm", 
            "Friendly", "Aggressive", "Engaged", "Distracted", 
            "Bored", "Frustrated","Dominant","Submissive"]

index = data.columns.tolist()
index = index[0:10] + ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + [c for c1 in constructs for c in ['left' + c1, 'right' + c1]]
data.columns=index
#data

## Pre-processing

For each left/right pair of constructs, compute the absolute difference and the sum (shifted to [-2, +2] interval).

This provides insight on the imbalance of the given construct between the children (difference), and the overall 'strength' of the construct in the clip (sum).

In [ ]:
for c in constructs:
    data["diff"+c] = abs(data["left" + c] - data["right" + c]) #calculating difference
    data["sum"+c] = data["left" + c] + data["right" + c] - 4 #calculating sum

Create 2 lists of columns names, one for diff/sum constructs, one for left/right constructs.

In [ ]:
columnsLeftRight=[]
columnsDiffSum=[]

for c in constructs:
    columnsLeftRight.append("left" + c)
    columnsLeftRight.append("right" + c)
    
    columnsDiffSum.append("diff" + c)
    columnsDiffSum.append("sum" + c)


# by default, work with differences & sum for each constructs
selectedColumns=columnsDiffSum

# work with differences & sum and the four questions about group dynamics
allQuestionsDiffSum = ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + columnsDiffSum

# work with left/right ratings and the four questions about group dynamics i.e. raw ratings
allQuestionsLeftRight = ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + columnsLeftRight

**Define several useful 'partial' views of the data.** <br>
Dataframes for each condition. <br>
Responses to left/right questions as dataframe and as array. <br>
Clip names. <br>
Mean ratings per clip.

In [ ]:
#FULL-SCENE DATA

fullscene_df = data[data["condition"]==2] # dataframe showing full scene data only

# the responses to the 26 left/right Likert-scale questions
fullscene_ratings_df = fullscene_df[selectedColumns].astype(float)
fullscene = fullscene_ratings_df.values # the underlying numpy array, needed for clustering

# clip names
fullscene_labels = fullscene_df["clipId"].values

# mean ratings per clip
fullscene_means = fullscene_df.groupby(["clipId"]).mean()[selectedColumns]

In [ ]:
#MOVEMENT-ALONE DATA

move_df=data[data["condition"]==1] # dataframe showing movement alone data only

# the responses to the 26 left/right Likert-scale questions
move_ratings_df=move_df[selectedColumns].astype(float)
move=move_ratings_df.values # the underlying numpy array, needed for clustering

# clip names
move_labels=move_df["clipId"].values

# mean ratings per clip
move_means=move_df.groupby(["clipId"]).mean()[selectedColumns]

# Demographics - Table 2.1

In [ ]:
move_age_mean = move_df['age'].mean()
move_age_max = max(move_df['age'])
move_age_min = min(move_df['age'])
move_per_male = ((len(move_df['gender'][move_df['gender']=='Male'])/4)/(len(move_df)/4))*100
move_per_female = ((len(move_df['gender'][move_df['gender']=='Female'])/4)/(len(move_df)/4))*100
move_per_usa = ((len(move_df['nationality'][move_df['nationality']=='American'])/4)/(len(move_df)/4))*100
move_per_eng = ((len(move_df['firstLang'][move_df['firstLang']=='English'])/4)/(len(move_df)/4))*100

full_age_mean = fullscene_df['age'].mean()
full_age_max = max(fullscene_df['age'])
full_age_min = min(fullscene_df['age'])
full_per_male = ((len(fullscene_df['gender'][fullscene_df['gender']=='Male'])/4)/(len(fullscene_df)/4))*100
full_per_female = ((len(fullscene_df['gender'][fullscene_df['gender']=='Female'])/4)/(len(fullscene_df)/4))*100
full_per_usa = ((len(fullscene_df['nationality'][fullscene_df['nationality']=='American'])/4)/(len(fullscene_df)/4))*100
full_per_eng = ((len(fullscene_df['firstLang'][fullscene_df['firstLang']=='English'])/4)/(len(fullscene_df)/4))*100

all_age_mean = data['age'].mean()
all_age_max = max(data['age'])
all_age_min = min(data['age'])
all_per_male = ((len(data['gender'][data['gender']=='Male'])/4)/(len(data)/4))*100
all_per_female = ((len(data['gender'][data['gender']=='Female'])/4)/(len(data)/4))*100
all_per_usa = ((len(data['nationality'][data['nationality']=='American'])/4)/(len(data)/4))*100
all_per_eng = ((len(data['firstLang'][data['firstLang']=='English'])/4)/(len(data)/4))*100

In [ ]:
Percent_Correct_Frames = pd.DataFrame({'N': [len(move_df)/4, len(fullscene_df)/4, (len(data)/4)],
                                      'Mean Age (Range)': [f'{move_age_mean:.2f}'+' ('+f'{move_age_min}'+'-'+f'{move_age_max}'+')', 
                                                           f'{full_age_mean:.2f}'+' ('+f'{full_age_min}'+'-'+f'{full_age_max}'+')', 
                                                           f'{all_age_mean:.2f}'+' ('+f'{all_age_min}'+'-'+f'{all_age_max}'+')'],
                                      'Gender %M %F': [f'{move_per_male:.2f}'+'%'+', '+f'{move_per_female:.2f}'+'%',
                                                      f'{full_per_male:.2f}'+'%'+', '+f'{full_per_female:.2f}'+'%',
                                                      f'{all_per_male:.2f}'+'%'+', '+f'{all_per_female:.2f}'+'%'],
                                      '% American': [f'{move_per_usa:.0f}'+'%',
                                                    f'{full_per_usa:.0f}'+'%',
                                                    f'{all_per_usa:.0f}'+'%'],
                                      '% English First Language': [f'{move_per_eng:.0f}'+'%',
                                                    f'{full_per_eng:.0f}'+'%',
                                                    f'{all_per_eng:.0f}'+'%']})

Percent_Correct_Frames.set_index([pd.Index(['Movement-Alone', 'Full-Scene', 'Both'])])

# 2.3.1 Inter-Rater Agreement

Calculate Kirppendorff's alpha to look at how much participants in each condition agreed on their ratings for each clip (lighter color means higher agreement).

In [ ]:
import krippendorff

krip={}

# agreement for fullscene condition for all questions
for clipName, group in fullscene_df[["clipId"] + allQuestionsLeftRight].groupby(["clipId"]): # working with all raw ratings
    krip[clipName]=(krippendorff.alpha(group.values[:,1:].astype(int),level_of_measurement='interval'), group.shape[0])

# agreement for movement-alone condition for all questions
for clipName, group in move_df[["clipId"] + allQuestionsLeftRight].groupby("clipId"):
    krip[clipName]=krip[clipName] + (krippendorff.alpha(group.values[:,1:].astype(int),level_of_measurement='interval'), group.shape[0])

# create random ratings and calculate "baseline" agreement score
for clipName, group in move_df[["clipId"] + allQuestionsLeftRight].groupby("clipId"):
    ratings = group.values[:,1:].astype(int)
    ratings = np.random.randint(0,5,ratings.shape)
    krip[clipName]=krip[clipName] + (krippendorff.alpha(ratings,level_of_measurement='interval'), group.shape[0])

    
krippendorff_df=pd.DataFrame.from_dict(krip,orient="index", columns=["Fullscene alpha", "N", "Movement-Alone alpha", "N","Random ratings alpha", "N"])

show_heatmap(krippendorff_df[["Fullscene alpha", "Movement-Alone alpha", "Random ratings alpha"]].round(3), cmap="summer")

Question: did participants in the fullscene condition agree more in their ratings of each clip than participants in the movement-alone condition?

We test this by comparing the agreement scores in the fullscene vs movement-alone videos, using an paired samples T-test. 

In [ ]:
from scipy.stats import ttest_rel
from math import sqrt

fullscene_krip = krippendorff_df["Fullscene alpha"]
move_krip = krippendorff_df["Movement-Alone alpha"]

print('Mean Kripp Alpha Fullscene:', fullscene_krip.mean())
print('SD:', fullscene_krip.std())
print('Mean Kripp Alpha Movement:', move_krip.mean())
print('SD:', move_krip.std())

print('Paired Samples T-Test:', ttest_rel(fullscene_krip, move_krip))

cohens_d = (fullscene_krip.mean() - move_krip.mean()) / (sqrt((fullscene_krip.std() ** 2 + move_krip.std() ** 2) / 2))

print("Cohen's d:", cohens_d)

T-test comparing krippendorf's alpha in the movement alone condition against 0 to see if there is agreement.

In [ ]:
from scipy.stats import ttest_1samp
print('One Sample T-Test:', ttest_1samp(move_krip, 0))

cohens_d = (move_krip.mean()-0) / move_krip.std()

print("Cohen's d:", cohens_d)

# 2.3.2 Automatic Labelling of Internal States

Multi-label classification using k-Nearest Neighbours (k=3). 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer



training_ground_truth = { '01': ['Aggressive'],
                         '02': ['Excited', 'Aggressive', 'Aimless'],
                         '03': ['Excited', 'Fun'],
                         '04': ['Cooperative'],
                         '05': ['Bored', 'Aimless'],
                         '06': ['Cooperative'],
                         '07': ['Dominant'],
                         '08': ['Bored', 'Fun'],
                         '09': ['Cooperative'],
                         '10': ['Cooperative', 'Dominant'],
                         '11': ['Cooperative', 'Dominant'],
                         '12': ['Aggressive', 'Aimless'],
                         '13': ['Excited', 'Aggressive', 'Aimless'],
                         '14': ['Aggressive'],
                         '15': ['Dominant'],
                         '16': ['Cooperative', 'Dominant'],
                         '17': ['Excited', 'Aggressive'],
                         '18': ['Aggressive', 'Dominant'],
                         '19': ['Dominant'],
                         '20': ['Excited']}

mlb = MultiLabelBinarizer()
mlb.fit(training_ground_truth.values())

def create_datasets(training=data, testing=None, cols=allQuestionsDiffSum, test_size=0.2, use_clip_id_as_label=False, random_labels=False, random_state=None):
    """Returns a training dataset and training labels, and a testing dataset and testing labels.
    
    If testing is None, it randomly splits the training dataframe (at test_size).
    """


    if testing is None:
        
        if use_clip_id_as_label:
            labels = list(training["clipId"].map(int))
        else:
            labels = []
            for id in training["clipId"]:
                labels.append(training_ground_truth[id])

        data = training[cols].values

        training_data, testing_data, training_labels, testing_labels = train_test_split(data, labels, test_size=test_size, random_state=random_state)

        if not use_clip_id_as_label:
            
            training_labels, testing_labels = mlb.transform(training_labels), mlb.transform(testing_labels)
            
            if random_labels:
                for labels in training_labels:
                    np.random.shuffle(labels)                 
                np.random.shuffle(training_labels)             
            

        return training_data, testing_data, training_labels, testing_labels
    
    else:
        
        if use_clip_id_as_label:
            training_labels = list(training["clipId"].map(int))
            testing_labels = list(testing["clipId"].map(int))
        else:
            labels = []
            for id in training["clipId"]:
                labels.append(training_ground_truth[id])

            training_labels = mlb.transform(labels)

            labels = []
            for id in testing["clipId"]:
                labels.append(training_ground_truth[id])

            testing_labels = mlb.transform(labels)

            if random_labels:
                if random_labels:
                    for labels in training_labels:
                        np.random.shuffle(labels)                 
                    np.random.shuffle(training_labels) 

        
        training_data = training[cols].values
        testing_data = testing[cols].values

        return training_data, testing_data, training_labels, testing_labels

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

def train(training_data, training_labels):
    
    #clf = RandomForestClassifier()
    clf = KNeighborsClassifier(n_neighbors=3)
    #clf = ExtraTreeClassifier(random_state=0)


    clf.fit(training_data, training_labels)
    
    return clf

def predict(clf, testing_data, inverse_transform_labels=True):
    p = clf.predict(testing_data)
    if inverse_transform_labels:
        return mlb.inverse_transform(p) 
    else:
        return p

In [ ]:
import sklearn
import sklearn.metrics as metrics
    
def run_classification(training, 
                       testing=None, 
                       cols=allQuestionsDiffSum, 
                       use_clip_id_as_label=False, 
                       random_labels=False,
                       crossvalidation_iterations=50):
    """
    Metrics for multi-label classification coming form Sorower, Mohammad S. "A literature survey on algorithms for multi-label learning." Oregon State University, Corvallis (2010).
    """
    
    results = {"Accuracy": [],
               "Precision": [],
               "Recall": [],
               "F1-measure": []}              
    labels_f1 = []
    

    for x in range(crossvalidation_iterations):
               
        training_data, testing_data, training_labels, testing_labels = create_datasets(training=training, 
                                                                                       testing=testing, 
                                                                                       cols=cols, 
                                                                                       use_clip_id_as_label=use_clip_id_as_label,
                                                                                       random_labels=random_labels,
                                                                                       random_state = x)
        
        if x == 0:
            print("Shape of training data: %s" % str(training_data.shape))
            print("Shape of testing data: %s" % str(testing_data.shape))
        
        clf = train(training_data, training_labels)

        pred_labels = predict(clf, testing_data, inverse_transform_labels = not use_clip_id_as_label)

        
        at_least_one = 0
        at_least_one_no_incorrect = 0
        
        if not use_clip_id_as_label:
            
            nb_classes = len(mlb.classes_)
            
            
            labels_f1.append(dict(zip(mlb.classes_, metrics.f1_score(testing_labels, mlb.transform(pred_labels), average=None))))
            
            results["Accuracy"].append(metrics.accuracy_score(testing_labels, mlb.transform(pred_labels)))
            results["Recall"].append(metrics.recall_score(testing_labels, mlb.transform(pred_labels), average='weighted'))    
            results["Precision"].append(metrics.precision_score(testing_labels, mlb.transform(pred_labels), average='weighted'))    
            results["F1-measure"].append(metrics.f1_score(testing_labels, mlb.transform(pred_labels), average='weighted'))    
            
            
            
            testing_labels = mlb.inverse_transform(testing_labels)
            
            exact = 0
            accuracy = 0
            precision = 0
            recall = 0
            f1_measure = 0
            
            for actual, pred in zip(testing_labels, pred_labels):
                
                pred = set(pred)
                actual = set(actual)
                
                if len(pred) == 0: continue
                    
                if pred == actual:
                    #print("%s <-> %s" % (actual, pred))
                    exact += 1
                    
                intersection = pred.intersection(actual)
                union = pred.union(actual)

                #accuracy += float(len(intersection)) / len(union)
                #precision += float(len(intersection)) / len(pred)
                #recall += float(len(intersection)) / len(actual)
                #f1_measure += 2 * float(len(intersection)) / (len(pred) + len(actual))
                
            
            #results["exact"].append(float(exact) / len(testing_labels))
            #results["accuracy"].append(accuracy / len(testing_labels))
            #results["precision"].append(precision / len(testing_labels))
            #results["recall"].append(recall / len(testing_labels))
            #results["f1_measure"].append(f1_measure / len(testing_labels))
            
            
            
        else: # use_clip_id_as_label = True
            # does not make much sense as at_least_one & at_least_one_no_incorrect are the same as 'exact'
            pass

    return pd.DataFrame(results), pd.DataFrame(labels_f1)

Analysis of the significance of the classification results, by computing a permutation-based p-value.

*This method is based on Ojala and Garriga 2010 "Permutation Tests for Studying Classifier Performance".*

In [ ]:
def compute_p_value_permutation(dataset, testing=None, k=10, crossvalidation_iterations=50):
    
    pvalues = []
    
    for x in range(crossvalidation_iterations):
        times_baseline_worst = 0

        training_data, testing_data, training_labels, testing_labels = create_datasets(training=dataset,
                                                                                       testing=testing,
                                                                                       random_state=x)

        # train the classifier
        clf = train(training_data, training_labels)

        # baseline prediction
        pred_labels = predict(clf, testing_data, inverse_transform_labels=False)

        baseline_error = 1 - metrics.f1_score(testing_labels, pred_labels, average='weighted')

        for i in range(k):
            testing_labels = np.random.permutation(testing_labels)

            error = 1 - metrics.f1_score(testing_labels, pred_labels, average='weighted')

            if error <= baseline_error:
                times_baseline_worst += 1

        pvalues.append((times_baseline_worst + 1)/float(k + 1))
    return pd.Series(pvalues)

### Fullscene classification - 80%/20% split - multi-label

In [ ]:
results, labels_f1_fs = run_classification(fullscene_df, crossvalidation_iterations=300)

First look at accuracy, precision, recall and f1 scores for overall performance on every label

In [ ]:
results.mean(axis = 0) 

We also computed F1 score for each classification label

In [ ]:
labels_f1_fs.mean(axis = 0) 

In [ ]:
compute_p_value_permutation(fullscene_df, k=100, crossvalidation_iterations=300).mean()

### Fullscene classification - 80%/20% split - multi-label - CHANCE level

The chance level is computed by associating random labels to the testing samples (still following the same distribution of labels as found in the original dataset).

In [ ]:
results_chance, labels_f1_fschance = run_classification(fullscene_df, random_labels=True, crossvalidation_iterations=300)

In [ ]:
results_chance.mean(axis = 0) 

In [ ]:
labels_f1_fschance.mean(axis = 0) 

### Fullscene training; movement alone testing - multi-labels

In [ ]:
results, labels_f1_ma = run_classification(fullscene_df, testing=move_df, crossvalidation_iterations=300)

In [ ]:
results.mean(axis = 0) 

In [ ]:
labels_f1_ma.mean(axis = 0) 

In [ ]:
compute_p_value_permutation(fullscene_df, testing=move_df, k=100, crossvalidation_iterations=300).mean()

### Fullscene training; movement alone testing - multi-labels - CHANCE level

The chance level is computed by associating random labels to the testing samples (still following the same distribution of labels as found in the original dataset).

In [ ]:
results, labels_f1_machance = run_classification(fullscene_df, testing=move_df, random_labels=True, crossvalidation_iterations=300)

In [ ]:
results.mean(axis = 0) 

In [ ]:
labels_f1_machance.mean(axis = 0) 

### Figure of mean F1-score for each label in each condition

In [ ]:
fs = labels_f1_fs.describe()
ma = labels_f1_ma.describe()

fschance = labels_f1_fschance.describe()
machance = labels_f1_machance.describe()

In [ ]:
fs_mean = fs.iloc[[1]].T
ma_mean = ma.iloc[[1]].T

f1_mean = pd.concat([fs_mean['mean'], ma_mean['mean']], axis=1, keys=['fullscene', 'movement alone'])
f1_mean
ax = f1_mean.plot.bar(rot=0, figsize=(10,5), color=['black', 'grey']) #plot
ax.set_ylabel("Mean F1 Score", fontsize=16)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.legend(fontsize=14)
ax.set_ylim(0,1)

ax.figure.savefig('../Figs/f1labelscore.png')

In [ ]:
fs_mean_chance = fschance.iloc[[1]].T
ma_mean_chance = machance.iloc[[1]].T
f1_mean_chance = pd.concat([fs_mean['mean'], fs_mean_chance['mean'], ma_mean['mean'], ma_mean_chance['mean']], axis=1, keys=['Fullscene', 'Fullscene Chance', 'Movement Alone', 'Movement Alone Chance'])

(f1_mean_chance).round(3).transpose()

# 2.3.3 Factor Analysis

Exploratory Factor Analysis examining what latent constructs underlie particiants' responses in each condition.

The Python factor_analyzer module is a port of EFA from the R' psych package.

In [ ]:
variable, total = factor_analyzer.factor_analyzer.calculate_kmo(fullscene_ratings_df)
print('KMO statistic Fullscene: ', total)
      
variable, total = factor_analyzer.factor_analyzer.calculate_kmo(move_ratings_df)
print('KMO statistic Movement: ', total)

In [ ]:
print('Bartlett Test Fullscene: ', factor_analyzer.factor_analyzer.calculate_bartlett_sphericity(fullscene_ratings_df))
print('Bartlett Test Movement: ', factor_analyzer.factor_analyzer.calculate_bartlett_sphericity(move_ratings_df))

In [ ]:
rotation = 'promax'

nb_factors=3

efa_fullscene = factor_analyzer.FactorAnalyzer()
efa_fullscene.analyze(fullscene_ratings_df, nb_factors, rotation=rotation)
fullscene_loadings=efa_fullscene.loadings

efa_move = factor_analyzer.FactorAnalyzer()
efa_move.analyze(move_ratings_df, nb_factors, rotation=rotation)
move_loadings=efa_move.loadings

In [ ]:
efa_fullscene.get_factor_variance() #variance explained by each construct for fullscene data

In [ ]:
efa_move.get_factor_variance() #variance explained by each construct for movement-alone data

Comparing the loadings for the *fullscene* vs the *movement alone* data show that the first three factors are highly correlated. **This shows that, using factor analysis, we have uncovered latent constructs that are used by participants to describe the clips in both *fullscene* and *movement alone* conditions**.

In [ ]:
# merge loadings into one dataframe, movement alone and fullscene side-by-side
loadings=pd.concat([fullscene_loadings, move_loadings], keys=["fullscene","movement alone"], axis=1)
loadings=loadings.swaplevel(0,1,1).sort_index(1)

from scipy.stats import pearsonr

print("Pearson correlation between factors 'fullscene' vs 'movement alone'")
for i in range(1, nb_factors+1):
    r, p=pearsonr(loadings["Factor%d" % i]["fullscene"].values, loadings["Factor%d" % i]["movement alone"].values)
    print("Factor %d: r=%f, p=%f" % (i,r,p)) 
    

show_heatmap(loadings[abs(loadings)>=0.3])

## Social Expressivness of the EFA Embeddings

### EFA embeddings

We can use the EFA space as a 'better' space to represent our clips, where the latent, composite constructs correspond to the main axis:

In [ ]:
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

nb_of_factors=3
fullscene_efa = np.dot(fullscene,fullscene_loadings.values[:,:nb_of_factors])
fullscene_means_efa = np.dot(fullscene_means,fullscene_loadings.values[:,:nb_of_factors])
move_efa = np.dot(move,fullscene_loadings.values[:,:nb_of_factors])
move_means_efa = np.dot(move_means,fullscene_loadings.values[:,:nb_of_factors])

move_pure_efa = np.dot(move,move_loadings.values[:,:nb_of_factors])
move_pure_means_efa = np.dot(move_means,move_loadings.values[:,:nb_of_factors])


plot_embedding(fullscene_efa, fullscene_labels,fullscene_means_efa, fullscene_means.index, title="EFA-space embedding of the fullscene data", three_d=True)
plot_embedding(move_efa, move_labels,move_means_efa, move_means.index, title="EFA-space embedding of the movement alone data (EFA on fullscene data)", three_d=False)
plot_embedding(move_pure_efa, move_labels,move_pure_means_efa, move_means.index, title="EFA-space embedding of the movement alone data (EFA on movement alone data)", three_d=False)


Interestingly, even if the EFA factors are quite similar, the distances between the same clips in fullscene vs movement alone data are high in the EFA space:

In [ ]:
distances_efa=pd.DataFrame(np.power(np.sum(np.power(move_means_efa - fullscene_means_efa, 2), axis=1), 0.5), index=move_means.index, columns=["distance_efa"])

print("Mean distance:\n%s" % distances_efa.mean(axis=0))
show_heatmap(distances_efa, cmap="summer_r")

Adding the EFA projections to the original dataframes:

In [ ]:
fullscene_df["efa1"] = pd.Series(fullscene_efa[:,0], index=fullscene_df.index)
fullscene_df["efa2"] = pd.Series(fullscene_efa[:,1], index=fullscene_df.index)
fullscene_df["efa3"] = pd.Series(fullscene_efa[:,2], index=fullscene_df.index)
move_df["efa1"] = pd.Series(move_efa[:,0], index=move_df.index)
move_df["efa2"] = pd.Series(move_efa[:,1], index=move_df.index)
move_df["efa3"] = pd.Series(move_efa[:,2], index=move_df.index)

We can then re-classify the clips, comparing the performance of the original 26-dimensional ratings to the 3-dimensional EFA-space projections (*still using a 300-fold cross-validation)*:

In [ ]:
nb_iterations = 300

print("Fullscene, 80%/20%...")
results_fullscene,labels_f1_fullscene = run_classification(fullscene_df, crossvalidation_iterations=nb_iterations)
print("Fullscene, 80%/20%, EFA space...")
results_fullscene_efa,labels_f1_fullscene_efa = run_classification(fullscene_df, cols=["efa1", "efa2", "efa3"], crossvalidation_iterations=nb_iterations)
print("Fullscene, chance level...")
results_fullscene_chance,labels_f1_fullscene_chance = run_classification(fullscene_df, random_labels=True, crossvalidation_iterations=nb_iterations)
print("Fullscene, 80%/20%, sanity check [input cols=['age']]...")
results_fullscene_age,labels_f1_fullscene_age = run_classification(fullscene_df, cols=["age"], crossvalidation_iterations=nb_iterations)

print("Fullscene vs skeletons...")
results_fullscene_move,labels_f1_move = run_classification(fullscene_df, testing=move_df, crossvalidation_iterations=nb_iterations)
print("Fullscene vs skeletons, EFA space...")
results_fullscene_move_efa,labels_f1_move_efa = run_classification(fullscene_df, testing=move_df, cols=["efa1", "efa2", "efa3"], crossvalidation_iterations=nb_iterations)
print("Fullscene vs skeletons, chance level...")
results_fullscene_move_chance,labels_f1_move_chance = run_classification(fullscene_df, testing=move_df, random_labels=True, crossvalidation_iterations=nb_iterations)

collated_results = pd.DataFrame({"Full-scene, EFA space": results_fullscene_efa.mean(),
                                 "Full-scene": results_fullscene.mean(),
                                 "Full-scene, chance": results_fullscene_chance.mean(),
                                 #"Full-scene-80-20-sanity-check": results_fullscene_age.mean(),
                                 "Movement-alone, EFA space": results_fullscene_move_efa.mean(),
                                 "Movement-alone": results_fullscene_move.mean(),
                                 "Movement-alone, chance": results_fullscene_move_chance.mean()})
labels_f1 = pd.concat({"Full-scene, EFA space": labels_f1_fullscene_efa,
                       "Full-scene": labels_f1_fullscene,
                       "Full-scene, chance": labels_f1_fullscene_chance,
                       #"fullscene-80-20-sanity-check": labels_f1_fullscene_age,
                       "Movement-alone, EFA": labels_f1_move_efa,
                       "Movement-alone": labels_f1_move,
                       "Movement-alone, chance": labels_f1_move_chance}, axis=1)

In [ ]:
collated_results.round(3)

In [ ]:
fullscene = labels_f1.loc[:, 'Full-scene']
fullsceneEFA = labels_f1.loc[:, 'Full-scene, EFA space']
fullsceneChance = labels_f1.loc[:, 'Full-scene, chance']
movement = labels_f1.loc[:, 'Movement-alone']
movementEFA = labels_f1.loc[:, 'Movement-alone, EFA']
movementChance = labels_f1.loc[:, 'Movement-alone, chance']

collated_labels = pd.DataFrame([fullsceneEFA.mean(axis=0)])
collated_labels = collated_labels.append([fullscene.mean(axis=0)],ignore_index=True)
collated_labels = collated_labels.append([fullsceneChance.mean(axis=0)],ignore_index=True)
collated_labels = collated_labels.append([movementEFA.mean(axis=0)],ignore_index=True)
collated_labels = collated_labels.append([movement.mean(axis=0)],ignore_index=True)
collated_labels = collated_labels.append([movementChance.mean(axis=0)],ignore_index=True)

collated_labels.rename(index={0:'Fullscene, EFA',1:'Fullscene',2:'Chance',
                              3:'Movement alone, EFA',4:'Movement alone',5:'Chance'}, inplace=True)
collated_labels